<a href="https://colab.research.google.com/github/DallasAutumn/datawhale_salons/blob/master/PyTorch_s8_task7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch-s8-task7 手写数字识别

In [0]:
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset


import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision

import matplotlib.pyplot as plt

import os

In [0]:
# Hyper parameters
EPOCH = 3
BATCH_SIZE = 32 #@param [8, 16, 32, 64]
LR = 0.001 #@param {type:"number"}
MNIST_ROOT = 'MNIST'
DOWNLOAD_MNIST = False if MNIST_ROOT in os.listdir() else True
TEST_SIZE = 2000

device = torch.device('cuda')
print(device, torch.cuda.get_device_name())

cuda Tesla T4


In [0]:
train_data = torchvision.datasets.MNIST(
        root=f'./{MNIST_ROOT}', 
        download=DOWNLOAD_MNIST, 
        train=True, 
        transform=torchvision.transforms.ToTensor()
    ) # 60000 range(0, 1)

In [0]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_data = torchvision.datasets.MNIST(root=f'./{MNIST_ROOT}', train=False)

with torch.no_grad():
    test_x = Variable(torch.unsqueeze(test_data.data, dim=1)).type(torch.cuda.FloatTensor)[:TEST_SIZE] / 255.
    test_y = test_data.targets.type(torch.cuda.LongTensor).squeeze()[:TEST_SIZE]
    test_y_for_acc = test_data.targets.numpy().squeeze()[:TEST_SIZE]
    test_x.to(device)
    test_y.to(device)



In [0]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), # padding = (kernel_size - stride) / 2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x) # (batch, 32, 7, 7)
        x = x.view(x.size(0), -1) # flatten (batch, 32*7*7)
        output = self.out(x)
        return output

cnn = CNN()
cnn.to(device)
print(cnn)


CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [0]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

# training and optimizing
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x).to(device)
        b_y = Variable(y).to(device)

        output = cnn(b_x)
        loss = loss_func(output, b_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y_for_acc = torch.max(test_output, 1)[1].cpu().data.numpy().squeeze()
            accuracy = sum(pred_y_for_acc == test_y_for_acc) / test_y_for_acc.size
            print('Epoch ', epoch, '| train loss : %.4f' % loss.item(), '| test accuracy', accuracy)

Epoch  0 | train loss : 2.3259 | test accuracy 0.1305
Epoch  0 | train loss : 0.7036 | test accuracy 0.696
Epoch  0 | train loss : 0.5569 | test accuracy 0.8845
Epoch  0 | train loss : 0.3729 | test accuracy 0.886
Epoch  0 | train loss : 0.1068 | test accuracy 0.9235
Epoch  0 | train loss : 0.0891 | test accuracy 0.9365
Epoch  0 | train loss : 0.1185 | test accuracy 0.9455
Epoch  0 | train loss : 0.1072 | test accuracy 0.946
Epoch  0 | train loss : 0.0192 | test accuracy 0.959
Epoch  0 | train loss : 0.0288 | test accuracy 0.96
Epoch  0 | train loss : 0.1034 | test accuracy 0.9595
Epoch  0 | train loss : 0.0920 | test accuracy 0.9565
Epoch  0 | train loss : 0.0595 | test accuracy 0.963
Epoch  0 | train loss : 0.1826 | test accuracy 0.966
Epoch  0 | train loss : 0.0305 | test accuracy 0.969
Epoch  0 | train loss : 0.0452 | test accuracy 0.9665
Epoch  0 | train loss : 0.1402 | test accuracy 0.972
Epoch  0 | train loss : 0.1787 | test accuracy 0.9695
Epoch  0 | train loss : 0.0258 | test 